In [ ]:
from google.colab import drive
drive.mount('/content/drive')

basic_path = '/content/drive/MyDrive/test/'

Mounted at /content/drive


In [ ]:
# 라이브러리 import
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

In [ ]:
def get_device():
    return torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def get_mean_std(channel, training_dataset):
  if channel.lower() is 'rgb':
    mean_rgb = [np.mean(x.numpy(), axis=(1, 2)) for x,_ in training_dataset]
    std_rgb = [np.std(x.numpy(), axis=(1, 2)) for x,_ in training_dataset]

    mean_r = np.mean([m[0] for m in mean_rgb])
    mean_g = np.mean([m[1] for m in mean_rgb])
    mean_b = np.mean([m[2] for m in mean_rgb])

    std_r = np.mean([s[0] for s in std_rgb])
    std_g = np.mean([s[1] for s in std_rgb])
    std_b = np.mean([s[2] for s in std_rgb])
    return [mean_r, mean_g, mean_b], [std_r, std_g, std_b]
  else:
    mean = [np.mean(x.numpy(), axis=(1, 2)) for x,_ in training_dataset]
    std = [np.std(x.numpy(), axis=(1, 2)) for x,_ in training_dataset]

    return np.mean([m[0] for m in mean]), np.mean([s[0] for s in std])

### ResNet 구현

In [ ]:
class BottleNeckBlock(nn.Module):
  def __init__(self, input_channel_size, output_channel_size, stride):
    super(BottleNeckBlock, self).__init__()
    self.residual_func = nn.Sequential(
            nn.Conv2d(in_channels=input_channel_size, out_channels=output_channel_size, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(output_channel_size),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=output_channel_size, out_channels=output_channel_size, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(output_channel_size),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=output_channel_size, out_channels=(output_channel_size * 4), kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(output_channel_size * 4),
        )
    self.myReLU = nn.ReLU()
    self.shortcut = nn.Sequential()
    if (stride != 1) or (input_channel_size != (output_channel_size * 4)):
      self.shortcut = nn.Sequential(
            nn.Conv2d(input_channel_size, (output_channel_size * 4), kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(output_channel_size * 4)
        )
  
  def forward(self, x):
    x = self.residual_func(x) + self.shortcut(x)
    x = self.myReLU(x)
    return x

In [ ]:
class MyResNet(nn.Module):
  def __init__(self, size_of_channel, number_of_class):
    super(MyResNet, self).__init__()
    self.base_input_channel = 64
    self.conv1 = nn.Sequential(
        nn.Conv2d(size_of_channel, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
    )
    self.conv2 = self.__generate_bottleneck_layer__(64, number_of_layer=3, base_stride=1)
    self.conv3 = self.__generate_bottleneck_layer__(128, number_of_layer=4)
    self.conv4 = self.__generate_bottleneck_layer__(256, number_of_layer=6)
    self.conv5 = self.__generate_bottleneck_layer__(512, number_of_layer=3)

    self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc = nn.Linear(2048, number_of_class)

    self.__init_weights__()

  def __generate_bottleneck_layer__(self, output_channel_size, number_of_layer, base_stride=2):
    strides = [base_stride] + [1] * (number_of_layer - 1)
    layers = []
    for stride in strides:
        layer = BottleNeckBlock(self.base_input_channel, output_channel_size, stride)
        self.base_input_channel = output_channel_size * 4
        layers.append(layer)
    return nn.Sequential(*layers)
  
  def __init_weights__(self):
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
          nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.constant_(m.bias, 0)

  def forward(self, x):
    output = self.conv1(x)
    output = self.conv2(output)
    x = self.conv3(output)
    x = self.conv4(x)
    x = self.conv5(x)
    x = self.avg_pool(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    return x

### Train / Test 코드

In [ ]:
def train(device, dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      
      pred = model(X.cuda())
      loss = loss_fn(pred, y)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
      print('.', end='')
      if batch % 100 == 0:
          print()
          loss, current = loss.item(), batch*len(X)
          print(f'loss: {loss:>7f}   [{current:>5d}/{size:>5d}]')

In [ ]:
def test(device, dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            loss = loss_fn(pred, y).item()
            test_loss += loss
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n')

### Model - CIFAR10

In [ ]:
device = get_device()
print('Device:', device)

Device: cuda


In [ ]:
training_dataset_cifar10 = datasets.CIFAR10(
    root=basic_path + '/data', train=True, download=True, transform=transforms.ToTensor(),
)
test_dataset_cifar10 = datasets.CIFAR10(
    root=basic_path + '/data', train=False, download=True, transform=transforms.ToTensor(),
)

rgb_mean, rgb_std = get_mean_std('rgb', training_dataset_cifar10)
rgb_transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Resize(224),
  transforms.Normalize(mean=[rgb_mean], std=[rgb_std])
])
training_dataset_cifar10.transform = rgb_transform
test_dataset_cifar10.transform = rgb_transform

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
training_dataloader_cifar10 = DataLoader(training_dataset_cifar10, batch_size=64)
test_dataloader_cifar10 = DataLoader(test_dataset_cifar10, batch_size=64)

for X, y in test_dataloader_cifar10:
  print('Shape of X [N, C, H, W]:', X.shape)
  print('Shape of y:', y.shape, y.dtype)
  break

Shape of X [N, C, H, W]: torch.Size([64, 3, 224, 224])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
number_of_cifar10_classes = 10
model_cifar10 = MyResNet(3, number_of_cifar10_classes)
model_cifar10 = model_cifar10.to(device)

In [ ]:
# Cuda out of memory
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from torchsummary import summary

summary(model_cifar10, input_size=(3, 224, 224))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [16]:
for epoch in range(5):
  print(f'Epoch {epoch + 1}\n---------------------------------')
  train(device, training_dataloader_cifar10, model_cifar10, nn.CrossEntropyLoss(reduction='sum'),
        torch.optim.Adam(model_cifar10.parameters(),lr=0.001))
  test(device, test_dataloader_cifar10, model_cifar10, nn.CrossEntropyLoss(reduction='sum'))

Epoch 1
---------------------------------
.
loss: 159.762497   [    0/50000]
....................................................................................................
loss: 115.318932   [ 6400/50000]
....................................................................................................
loss: 105.587845   [12800/50000]
....................................................................................................
loss: 110.483849   [19200/50000]
....................................................................................................
loss: 104.279259   [25600/50000]
....................................................................................................
loss: 101.732597   [32000/50000]
....................................................................................................
loss: 88.263199   [38400/50000]
....................................................................................................
loss: 92.765923   [

In [17]:
dest_model_path = basic_path + 'resnet_models/model_cifar10_v2.pth'
print(dest_model_path)
torch.save(model_cifar10.state_dict(), dest_model_path)

/content/drive/MyDrive/test/resnet_models/model_cifar10_v2.pth
